# Imports and Factory Configuration

### 1. **Imports**
The script imports several modules required for GUI, network communication, and randomization.
- `tkinter`: Used to create the graphical user interface (GUI) for interaction and visualization.
- `scrolledtext`: A component from `tkinter` to display logs with a scrollable text area.
- `socket`: Used to establish UDP communication with the Robotino robots.
- `random`: Used to simulate random robot movements.
- `time`: Used to introduce delays in communication to mimic real-world network lag.

### 2. **Factory Configuration**
The factory is configured with details of:
- **Robots**: IP addresses, locations, chassis types, and battery levels for six Robotino robots.
- **Chargers**: Locations and compatibility with different Robotino models.
- **Stations**: Locations of workstations or areas where robots interact with the environment.

This configuration allows the system to visualize robot, charger, and station positions and track the state of each component.


In [1]:
import tkinter as tk
from tkinter import scrolledtext
import socket
import random
import time

# Factory configuration: Robotino, chargers, and stations (locations, types, etc.)
factory_config = {
    "robots": {
        "172.21.20.90": {"chassis_type": "model3", "location": [2, 5], "battery_level": 100},
        "172.21.21.90": {"chassis_type": "model3", "location": [6, 7], "battery_level": 100},
        "172.21.22.90": {"chassis_type": "model3", "location": [1, 8], "battery_level": 100},
        "172.21.23.90": {"chassis_type": "model3", "location": [3, 4], "battery_level": 100},
        "172.21.24.90": {"chassis_type": "model4", "location": [5, 1], "battery_level": 100},
        "172.21.25.90": {"chassis_type": "model4", "location": [4, 6], "battery_level": 100},
    },
    "chargers": {
        "Charger1": {"location": [0, 0], "chassis_type": "model3"},
        "Charger2": {"location": [8, 0], "chassis_type": "model3"},
        "Charger3": {"location": [0, 8], "chassis_type": "model3"},
        "Charger4": {"location": [8, 8], "chassis_type": "model3"},
        "Charger5": {"location": [4, 0], "chassis_type": "model4"},
        "Charger6": {"location": [0, 4], "chassis_type": "model4"},
    },
    "stations": {
        "1": {"type": "CP-F-RASS", "location": [-12.313, 4.249], "approach_location": 1},
        "2": {"type": "CP-F-RASS", "location": [-9.824, 4.267], "approach_location": 2},
        "3": {"type": "CP-F-RASS", "location": [-7.398, 4.286], "approach_location": 3},
    }
}

robotino_port = 13000  # UDP port for Robotino communication


# Distance Calculation

### **Function: `calculate_distance(coord1, coord2)`**
This function calculates the Euclidean distance between two points (2D coordinates) on a plane. It is essential for determining how far a robot is from a charger or station.

#### **Parameters**
- `coord1`: List or tuple of (x, y) coordinates for the first point.
- `coord2`: List or tuple of (x, y) coordinates for the second point.

#### **How It Works**
1. Subtracts x and y coordinates of the two points.
2. Squares the differences, sums them, and then takes the square root.

#### **Use Case**
This function is crucial for determining the closest charger to a robot.


In [2]:
def calculate_distance(coord1, coord2):
    return ((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2) ** 0.5


# Find Nearest Charger

### **Function: `find_nearest_charger(robot_location, chassis_type)`**
This function identifies the closest compatible charger for a specific robot.

#### **Parameters**
- `robot_location`: Current (x, y) position of the robot.
- `chassis_type`: The model of the robot (e.g., "model3" or "model4").

#### **How It Works**
1. Loops through all available chargers from the `factory_config`.
2. Checks if the charger's chassis type is compatible with the robot's chassis type.
3. Calculates the distance between the robot's location and the charger's location.
4. Selects the charger with the minimum distance and returns its identifier.

#### **Use Case**
This function allows robots to automatically locate the nearest charging station that matches their chassis type.


In [3]:
def find_nearest_charger(robot_location, chassis_type):
    nearest = None
    min_distance = float("inf")
    for charger, details in factory_config["chargers"].items():
        if details["chassis_type"] == chassis_type:
            distance = calculate_distance(robot_location, details["location"])
            if distance < min_distance:
                min_distance = distance
                nearest = charger
    return nearest


# Send Commands via UDP

### **Function: `send_command(command)`**
This function sends a command to a selected Robotino robot using UDP protocol.

#### **Parameters**
- `command`: The command to send (e.g., "GoTo [x, y]" or "DockToCharge").

#### **How It Works**
1. Retrieves the IP address of the selected robot from the GUI dropdown.
2. Sends the command to the robot using a UDP socket.
3. Logs the message in the GUI and waits for 0.5 seconds to simulate network delay.
4. Logs the robot's acknowledgment of the command.

#### **Error Handling**
If any error occurs during the transmission, it logs the error message.

#### **Use Case**
Used to control robot movement, docking, and other actions via network commands.


In [4]:
def send_command(command):
    ip = selected_ip.get()  # Get the selected robot IP from the dropdown
    try:
        message = f"{command}"
        udp_socket.sendto(message.encode(), (ip, robotino_port))  # Send command via UDP
        log_message(f"Sent to {ip}: {command}")  # Log the sent command
        time.sleep(0.5)  # Simulating communication delay
        log_message(f"Robotino {ip} acknowledged: {command}")  # Log acknowledgment
    except Exception as e:
        log_message(f"Error sending to {ip}: {e}")  # Log any error
